In [1]:
import glob
import pandas as pd

In [2]:
# Criterios de coincidencia de patrones para archivos de Excel en el directorio
filenames = 'Cuenca*.xlsx'

In [3]:
# Crea un diccionario vacío para almacenar los datos de cada archivo
dat = {}

# Recorra cada archivo de Excel que coincida con el patrón y lea sus datos en un marco de datos de pandas
for file in glob.glob(filenames):
    dat[file] = pd.read_excel(file)

In [4]:
# Access the data from each file using its file name as the key in the data_dict
data_new = []

for file, data in dat.items():
    data_new.append(data)

In [6]:
# Definir una función para aplicar a los datos y en la columna 'ESTACION' todos los valores tengan 3 digitos
for i, df in enumerate(data_new):
    def formatear_valor(valor):
        if valor < 10:
            return '00' + str(valor)
        elif valor < 100:
            return '0' + str(valor)
        else:
            return str(valor)

    # Aplicar la función a la tabla
    data_new[i]['ESTACION'] = data_new[i]['ESTACION'].apply(formatear_valor)

In [7]:
data_new

[       CUENCA ESTACION      FECHA    MAX    MIN
 0         100      633 2003-10-17  30.37  22.96
 1         100      633 2003-10-18  31.28  24.03
 2         100      633 2003-10-19  26.45  23.46
 3         100      633 2003-10-20  25.54  23.43
 4         100      633 2003-10-21  27.59  23.50
 ...       ...      ...        ...    ...    ...
 29327     100      655 2022-12-27  32.90  22.70
 29328     100      655 2022-12-28  31.50  22.50
 29329     100      655 2022-12-29  31.80  22.80
 29330     100      655 2022-12-30  32.70  23.10
 29331     100      655 2022-12-31  32.60  22.00
 
 [29332 rows x 5 columns],
       CUENCA ESTACION      FECHA    MAX    MIN
 0        200      002 2002-01-01  28.76  23.30
 1        200      002 2002-01-02  28.85  23.93
 2        200      002 2002-01-03  26.98  23.89
 3        200      002 2002-01-04  28.69  22.63
 4        200      002 2002-01-05  28.69  22.87
 ...      ...      ...        ...    ...    ...
 6977     200      009 2018-03-04  30.90  21.60

In [8]:
#Unir el numero de 'CUENCA' con 'ESTACION' para tener el numero de cada estacion
for i, df in enumerate(data_new):
    data_new[i]['ESTACION_NEW'] = data_new[i]['CUENCA'].astype(str) + data_new[i]['ESTACION'].astype(str)

In [9]:
#Eliminar las columnas que no son necesarias
for i, df in enumerate(data_new):
    data_new[i].pop('CUENCA')
    data_new[i].pop('ESTACION')

In [11]:
data_new

[           FECHA    MAX    MIN ESTACION_NEW
 0     2003-10-17  30.37  22.96       100633
 1     2003-10-18  31.28  24.03       100633
 2     2003-10-19  26.45  23.46       100633
 3     2003-10-20  25.54  23.43       100633
 4     2003-10-21  27.59  23.50       100633
 ...          ...    ...    ...          ...
 29327 2022-12-27  32.90  22.70       100655
 29328 2022-12-28  31.50  22.50       100655
 29329 2022-12-29  31.80  22.80       100655
 29330 2022-12-30  32.70  23.10       100655
 29331 2022-12-31  32.60  22.00       100655
 
 [29332 rows x 4 columns],
           FECHA    MAX    MIN ESTACION_NEW
 0    2002-01-01  28.76  23.30       200002
 1    2002-01-02  28.85  23.93       200002
 2    2002-01-03  26.98  23.89       200002
 3    2002-01-04  28.69  22.63       200002
 4    2002-01-05  28.69  22.87       200002
 ...         ...    ...    ...          ...
 6977 2018-03-04  30.90  21.60       200009
 6978 2018-03-05  32.20  22.30       200009
 6979 2018-03-06  32.50  22.80     

In [90]:
# Agrupar las filas según los valores de la columna "ESTACION_NEW"
my_list = list(range(0, 22))
porestacion=[]
for i in my_list:
    grupos = data_new[i].groupby('ESTACION_NEW')

    # Crear una tabla separada para cada grupo segun 'ESTACION_NEW'
    for ESTACION, grupo in grupos:
        # Obtener el DataFrame correspondiente a cada grupo
        tabla = grupos.get_group(ESTACION)
        porestacion.append(tabla) #Adjuntar los valores

In [91]:
NUMERO = porestacion.copy()
for i, df in enumerate(NUMERO):
    NUMERO[i] = NUMERO[i].iloc[0,3]

In [92]:
ESTACION = NUMERO.copy()
for i, df in enumerate(ESTACION):
    ESTACION[i] = 'Estacion' + NUMERO[i]

In [214]:
#Guardar cada tabla de cada estación en un CSV diferente
my_list = list(range(0, 154))
for a, i in zip(ESTACION,my_list):
    porestacion[i].to_csv(f'{a}.csv', index=True)

In [106]:
for i, df in enumerate(porestacion):
    porestacion[i] = porestacion[i].set_index('FECHA')

In [107]:
# Encontrar los valores de inicio y final de las fechas
my_list = list(range(0, 154))
porestacion_new=[]
for i in my_list:
    fecha_inicial = porestacion[i].index.min()
    fecha_final = porestacion[i].index.max()
    name = NUMERO[i]
    porestacion_new.append((fecha_inicial,fecha_final, name))

In [108]:
porestacion_new

[(Timestamp('2003-10-17 00:00:00'),
  Timestamp('2022-12-31 00:00:00'),
  '100633'),
 (Timestamp('2007-03-11 00:00:00'),
  Timestamp('2022-12-31 00:00:00'),
  '100641'),
 (Timestamp('2009-11-25 00:00:00'),
  Timestamp('2022-12-31 00:00:00'),
  '100643'),
 (Timestamp('2013-01-29 00:00:00'),
  Timestamp('2022-11-10 00:00:00'),
  '100649'),
 (Timestamp('2013-05-17 00:00:00'),
  Timestamp('2022-12-31 00:00:00'),
  '100651'),
 (Timestamp('2013-09-06 00:00:00'),
  Timestamp('2022-11-10 00:00:00'),
  '100653'),
 (Timestamp('2014-10-24 00:00:00'),
  Timestamp('2022-12-31 00:00:00'),
  '100655'),
 (Timestamp('2002-01-01 00:00:00'),
  Timestamp('2017-07-11 00:00:00'),
  '200002'),
 (Timestamp('2008-05-25 00:00:00'),
  Timestamp('2019-08-15 00:00:00'),
  '200007'),
 (Timestamp('2013-11-24 00:00:00'),
  Timestamp('2018-03-08 00:00:00'),
  '200009'),
 (Timestamp('2002-01-01 00:00:00'), Timestamp('2022-07-13 00:00:00'), '69633'),
 (Timestamp('2002-11-08 00:00:00'), Timestamp('2022-12-31 00:00:00'), 

In [109]:
#Guardar el rango de fechas en un archivo .txt
# Abrir el archivo en modo escritura
with open("rango_fechas.txt", "w") as archivo:
    # Escribir la lista en el archivo
    for valor in porestacion_new:
        archivo.write(str(valor) + "\n")

In [78]:
#Eliminar las columnas que no se necesitan
for i, df in enumerate(porestacion):
    porestacion[i].pop('ESTACION_NEW')

In [79]:
# Agrupar las filas según los valores de la columna "ESTACION_NEW"
my_list = list(range(0, 154))
TempMAX=[]
TempMIN=[]
for i in my_list:
    maxima = porestacion[i].copy()
    maxima.pop('MIN')
    minima = porestacion[i].copy()
    minima.pop('MAX')
    TempMAX.append(maxima)
    TempMIN.append(minima)

In [80]:
for i, df in enumerate(TempMAX):
    TempMAX[i] = TempMAX[i].set_index('FECHA')
    
for i, df in enumerate(TempMIN):
    TempMIN[i] = TempMIN[i].set_index('FECHA')

In [81]:
TempMAX

[              MAX
 FECHA            
 2003-10-17  30.37
 2003-10-18  31.28
 2003-10-19  26.45
 2003-10-20  25.54
 2003-10-21  27.59
 ...           ...
 2022-12-27  39.10
 2022-12-28  30.00
 2022-12-29  30.90
 2022-12-30  31.00
 2022-12-31  30.20
 
 [6656 rows x 1 columns],
               MAX
 FECHA            
 2007-03-11  34.71
 2007-03-12  34.91
 2007-03-13  35.18
 2007-03-14  35.05
 2007-03-15  34.71
 ...           ...
 2022-12-27  32.60
 2022-12-28  32.00
 2022-12-29  32.70
 2022-12-30  32.00
 2022-12-31  32.30
 
 [5355 rows x 1 columns],
               MAX
 FECHA            
 2009-11-25  24.72
 2009-11-26  24.76
 2009-12-13  32.75
 2009-12-14  33.43
 2009-12-15  32.99
 ...           ...
 2022-12-27  31.90
 2022-12-28  31.80
 2022-12-29  32.60
 2022-12-30  31.10
 2022-12-31  32.20
 
 [4765 rows x 1 columns],
               MAX
 FECHA            
 2013-01-29  33.71
 2013-01-30  32.79
 2013-01-31  34.00
 2013-02-01  33.63
 2013-02-02  33.20
 ...           ...
 2022-11-06  28.10
 202

In [96]:
#Obtener el promedio diario
TempMAX_mean = []
for i, df in enumerate(TempMAX):
    value = TempMAX[i].groupby(TempMAX[i].index.strftime("%m%d")).mean() 
    #groupby() se usa agrupando por mes y por día a la vez
    TempMAX_mean.append(value)
    
#Obtener el promedio diario
TempMIN_mean = []
for i, df in enumerate(TempMIN):
    value = TempMIN[i].groupby(TempMIN[i].index.strftime("%m%d")).mean() 
    #groupby() se usa agrupando por mes y por día a la vez
    TempMIN_mean.append(value)

In [97]:
#Seleccionar solo la columna que se quiere
for i, df in enumerate(TempMAX_mean):
    TempMAX_mean[i] = TempMAX_mean[i].iloc[:,0]

for i, df in enumerate(TempMIN_mean):
    TempMIN_mean[i] = TempMIN_mean[i].iloc[:,0]

In [98]:
#Volver un dataframe
TempMIN_mean = pd.DataFrame(TempMIN_mean)
TempMAX_mean = pd.DataFrame(TempMAX_mean)

In [99]:
#Transponer los valores
TempMIN_mean = TempMIN_mean.T
TempMAX_mean = TempMAX_mean.T

In [100]:
#Cambiar el nombre de las columnas
TempMIN_mean.columns = NUMERO
TempMAX_mean.columns = NUMERO

In [102]:
TempMIN_mean

,100633,100641,100643,100649,100651,100653,100655,200002,200007,200009,...,94019,96003,98075,98087,98091,98095,98097,98105,98107,98109
FECHA,,,,,,,,,,,,,,,,,,,,,
0101,22.102778,22.093333,21.534615,22.520000,22.000000,21.394444,23.078750,23.920000,25.758889,23.452,...,3.350,23.130000,16.219500,19.508889,3.322000,13.8420,17.738571,19.281250,16.550000,17.22
0102,22.192778,22.120000,21.835385,22.612500,22.257778,21.532222,22.956250,23.564286,25.774444,23.098,...,4.075,22.926667,16.523000,19.472222,3.109500,13.9650,17.802143,18.595000,16.683333,17.78
0103,22.032778,22.068333,21.513846,22.395000,22.138889,21.546667,22.788750,23.657143,25.607778,23.658,...,5.200,23.023333,16.517000,19.445789,3.416000,14.1995,17.562143,18.763750,17.450000,18.38
0104,21.968333,21.875833,21.596923,22.371250,22.205556,21.470000,22.351250,23.124286,25.117500,23.548,...,3.425,22.915556,16.365000,19.653684,3.066000,13.8995,17.617143,19.122500,16.916667,18.00
0105,22.272778,21.968333,21.793077,22.654286,22.132222,21.894444,22.858750,23.278571,25.326667,23.542,...,2.025,23.055556,16.596500,19.617895,3.525500,13.9350,17.797143,19.091250,16.450000,17.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,22.232222,22.121538,21.617857,22.622500,21.880000,21.281111,22.800000,23.685000,25.491111,23.020,...,4.140,22.875556,16.414737,20.157222,3.346316,13.8955,17.600000,19.054286,16.616667,17.62
1228,21.845556,21.618462,21.456429,22.436250,21.560000,20.890000,22.551111,23.428333,25.262222,22.886,...,4.100,22.827778,16.611053,20.055556,3.744737,14.1285,17.588000,19.431250,16.733333,17.68
1229,22.250556,22.407692,21.921429,22.498750,22.383000,21.493333,23.003333,23.641667,25.377778,22.924,...,4.880,23.194444,16.924211,19.870556,3.502632,14.3565,18.011333,18.805000,17.150000,18.16


In [110]:
#Cargar la lista de estaciones que contienen el rango de fechas
estaciones = pd.read_excel(r'/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observada/Temperatura/Lista de estaciones.xlsx')

In [111]:
estaciones

,NUMERO,NOMBRE,LATITUD NORTE,LONGITUD OESTE,ALTITUD (msnm),INICIO,FINAL,ESTACION
0,100633,FINCA EL PATIO. PUERTO JIMENEZ,"08º 36' 04.6""","83º25'52.2""",24,2003-10-17,2022-12-31,100633
1,100641,COOPEAGROPAL. LAUREL .,"08º 28' 17.6""","82º51'38.4""",38,2007-03-11,2022-12-31,100641
2,100643,RIO CLARO,"08º 40' 26.5""","83º03'43.2""",51,2009-11-25,2022-12-31,100643
3,100649,FUNDACION NEOTROPICA. RINCON DE OSA,"08º 42' 2.9""","83º30'49.4""",80,2013-01-29,2022-11-10,100649
4,100651,COTO 49. CIUDAD NEILY,"08º 37' 52.3""","82º58'00.7""",30,2013-05-17,2022-12-31,100651
...,...,...,...,...,...,...,...,...
149,98095,ALTAMIRA,"09º 01' 45.7""","83º00'28.7""",1370,2002-01-01,2022-12-31,98095
150,98097,MONTECARLO. PZ,"09º 21' 28""","83º36'03""",1005,2008-06-18,2022-12-31,98097
151,98105,PLANTA DE VOLCAN. BUENOS AIRES,"09º 12' 15""","83º27'2.49""",416,2013-11-22,2022-12-07,98105
152,98107,CAFETALERA EL INDIO. SAN VITO. PUNTARENAS,"08º 49' 57.9""","82º57'10.5""",1027,2016-12-16,2022-11-10,98107
